# 循迹

如果您已经完成了避免碰撞示例，那么您应该熟悉以下三个步骤

1. 数据采集
2. 训练
3. 部署方式

在此notebook中，我们将做同样的事情！除了分类之外，您将学习另一种基本技术，**回归**，我们将用它使JetBot遵循一条道路（或实际上是任何路径或目标点）。

将JetBot放置在路径上的不同位置（与中心的偏移，不同角度等）
请记住，从避免碰撞的角度来看，数据变化是关键！

显示来自机器人的实时视频
使用游戏手柄控制器，在图像上放置一个“绿点”，该绿点对应于我们希望机器人移动的目标方向。
将绿点的X，Y值以及机器人相机的图像存在一起。
然后，在训练notebook中，我们将训练神经网络来预测标签的X，Y值。在现场演示中，我们将使用预测的X，Y值来计算近似的转向值（这不是“精确”的角度，因为这需要进行图像校准，但它与角度大致成比例，因此我们的控制器可以正常工作）。

那么，如何确定本示例中目标的确切位置？我们认为可能会有所帮助的指导

1. 看一下摄像机的实时视频
2. 想象一下机器人应该遵循的路径（尝试估算出可以避免偏离路径所需的距离）。
3. 将目标沿着此路径尽可能远地放置，以使机器人可以直奔目标而不会“偏离”道路。
> 例如，如果我们处在非常直的道路上，则可以将其放置在地平线上。如果我们位于急转弯处，则可能需要将其放置在离机器人更近的位置，以免超出边界。

假设我们的深度学习模型按预期工作，则这些标签准则应确保以下各项：

1. 机器人可以安全地直接朝目标行进（不会越界等）
2. 目标将沿着我们想象的道路不断前进
我们得到的是沿着我们期望的轨迹移动，深度学习决定把目标放在哪里，jetbot会紧随其后:)

### 标记示的例视频
执行代码块以查看如何标记图像的示例。该模型仅需123张图像即可工作，可根据实际效果来增加采集量:)

In [1]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.bilibili.com/video/BV17f4y127pn/" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


### 导入python库

让我们开始导入用于“数据收集”目的的所有必需库。我们将主要使用OpenCV来可视化并保存带有标签的图像。uuid，datetime之类的库用于图像命名。

In [2]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### 实时显示摄像机反馈

首先，让我们像在遥控notebook中一样初始化并显示我们的相机。

我们使用JetBot的Camera类来启用CSI MIPI摄像机。我们的神经网络将224x224像素的图像作为输入。我们将相机设置为该大小，以最大程度地减少数据集文件的大小（我们已经测试了它适用于此任务）。在某些情况下，以较大的图像大小收集数据会更好，然后再缩小为所需大小。

In [3]:
camera = Camera.instance(width=224, height=224, fps=10)

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.0, description='y', max=1.0, min=-1.0, step=0.001)

### 创建游戏手柄控制器

此步骤类似于“远程操作”任务。在此任务中，我们将使用游戏手柄控制器标记图像。

我们要做的第一件事是创建Controller组件的实例，我们将使用它为图像标记“ x”和“ y”值。Controller组件带有一个索引参数，该参数指定控制器的编号。如果您连接了多个控制器，或者某些游戏手柄显示为多个控制器时很有用。要确定您正在使用的控制器的索引。

访问http://html5gamepad.com.
按下您正在使用的游戏板上的按钮。记住响应按钮按下的游戏板索引。下一步，我们将使用该索引创建并显示控制器。

In [7]:
controller = widgets.Controller(index=0)

display(controller)

Controller()

### 将Gamepad Controller和标记图像相关联

现在，即使我们已经连接了游戏手柄，我们仍未将控制器关联到标记图像！我们将使用dlink函数将其连接到左右垂直轴。与link功能不同，dlink函数不允许我们在源和目标之间相互转换。

In [8]:
widgets.jsdlink((controller.axes[2], 'value'), (x_slider, 'value'))
widgets.jsdlink((controller.axes[3], 'value'), (y_slider, 'value'))

DirectionalLink(source=(Axis(value=0.0), 'value'), target=(FloatSlider(value=0.0, description='y', max=1.0, mi…

### 收集数据

以下代码块将显示实时图像，以及我们已保存的图像数。我们将目标X，Y值通过以下方式存储

1. 将绿点放在目标上
2. 在手柄上按[4]保存
这会将文件存储在文件夹``dataset_xy``中，文件名为
``xy_<x value>_<y value>_<uuid>.jpg``

训练时，我们加载图像并解析文件名中的x，y值

In [9]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

for b in controller.buttons:
    b.unobserve_all()

count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot(change):
    if change['new']:
        uuid = xy_uuid(x_slider.value, y_slider.value)
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(image_widget.value)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

controller.buttons[4].observe(save_snapshot, names='value')

display(widgets.VBox([
    target_widget,
    count_widget
]))

Directories not created becasue they already exist


### 下一步

收集到足够的数据后，我们需要将该数据复制到我们的GPU台式机或云机上进行训练。首先，我们可以调用以下终端命令将数据集文件夹压缩为一个zip文件。

> 如果您正在使用JetBot本身进行训练，则可以跳过此步骤！

！前缀表示我们要将单元作为shell（或终端）命令运行。

下面zip命令中的-r标志表示递归，以便我们包含所有嵌套文件，-q标志表示隐藏，以便zip命令不打印任何输出

In [10]:
camera.stop()

In [11]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
camera.stop()
!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}

您应该在Jupyter Lab文件浏览器中看到一个名为road_following_ <Date＆Time> .zip的文件。您应该使用Jupyter Lab文件浏览器通过右键单击并选择下载来下载zip文件。